# Part 1  
---  
## Demultiplexing:   
---  
### 1) File Identification  

| File Name                    | ID      |  
|:----------------------------:|:-------:|  
| 1294_S1_L008_R1_001.fastq.gz | read1  |  
| 1294_S1_L008_R2_001.fastq.gz | index1 |  
| 1294_S1_L008_R3_001.fastq.gz | index2 |  
| 1294_S1_L008_R4_001.fastq.gz | read2  |  


### 2a) Python Code for Mean Q-score Analysis  
---  
Written by: Max Hills  
**See Mean Quality-Score Graphs and Tables Below**   

In [54]:
% matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np
def convert_phred(letter):
    """The function takes a single string character and converts it into the Illumina quality score."""
    phred_score = ord(letter)-33
    return phred_score


In [76]:
def avg_qs(fastq):
    """INPUT: fastq - A standard format fastq file of type str, readlen - The int length of the reads (in nucleotides)
    OUTPUT: A table of average q-scores at each position, A line-graph of average q-scores at each position"""
    with open(fastq,'r') as f:
        # Initialize a line-count.
        LC = 0
        # Initialize a read-count.
        RC = 0
        # Iterate through each line in the file.
        for line in f:        
            # Increment the line count.
            LC += 1        
            # If the line number is a multiple of 4, do the following...
            if LC%4 == 0:
                line = line.strip()
                readlen = len(line)
                if RC == 0:
                    all_qscores = np.zeros(readlen,dtype=int)
                # Increment the read-count.
                RC += 1
                # Initialize charIndex
                charIndex = 0
                for char in line:
                    # Get the decimal phred score at this position.
                    score = convert_phred(char)
                    # Store the score for the current position at its index within all_qscores. 
                    all_qscores[charIndex] += score
                    # Increment charIndex.
                    charIndex += 1
    mean_scores = all_qscores/RC
    table = fastq.replace('fastq','txt').replace('.gz','')
    graph = fastq.replace('fastq','png').replace('.gz','')
    with open(table, 'w') as t:
        # Print a header for the columns.
        print("# Base Pos\tMean Quality Score",file=t)
        print("===========\t==================",file=t)
        for i in range(readlen):
            template = "{}\t\t{:0.1f}"
            out = str(template.format(i,mean_scores[i]))
            print(out,file=t)
    # Initialize empty lists to hold x,y axis labels.
    x = []
    y = []
    ylow = math.floor(min(mean_scores))-2
    yhigh = math.ceil(max(mean_scores))+2
    # Create x labels stepwise by 2's.
    for n in range(0,readlen+1,2):
        x.append(n)
    # Create y labels stepwise by 1's.
    for n in range(ylow,yhigh):
        y.append(n)
    # Create the figure, with size and color specifications
    plt.figure(num=None, figsize=(16, 8), dpi=80, facecolor='w', edgecolor='k')
    # Plot the mean_scores, with a dark blue line.
    plt.plot(mean_scores, "darkblue")
    # Create the x,y tick marks
    plt.xticks(x)
    plt.yticks(y)
    # Label the x,y axes.
    plt.xlabel("Position of Base in Read")
    plt.ylabel("Mean Quality Score")
    # Set the range for the x,y axes.
    plt.axis([-1,(readlen+1), ylow, yhigh])
    # Title the plot
    plt.title("Average Quality Scores @ Base Position X")
    # Show the graph grid, then show the figure.
    plt.grid(True)
    plt.savefig(graph,format='png')

In [81]:
%%bash 
echo "Index 1 Mean Quality Scores By Nucleotide Position"
cat 1294*R2*.txt
echo "Read 1 Mean Quality Scores By Nucleotide Position"
cat 1294*R1*.txt
echo "Index 2 Mean Quality Scores By Nucleotide Position"
cat 1294*R3*.txt  
echo "Read 2 Mean Quality Scores By Nucleotide Position"
cat 1294*R4*.txt

Index 1 Mean Quality Scores By Nucleotide Position
# Base Pos	Mean Quality Score
===========	==================
0		30.98
1		31.37
2		35.29
3		35.88
4		35.98
5		39.35
6		39.10
7		38.97
Read 1 Mean Quality Scores By Nucleotide Position
# Base Pos	Mean Quality Score
===========	==================
0		31.26
1		31.44
2		35.33
3		35.93
4		36.10
5		39.57
6		39.74
7		39.82
8		40.01
9		40.07
10		40.10
11		40.12
12		40.06
13		40.07
14		40.07
15		40.07
16		39.98
17		40.01
18		40.02
19		40.01
20		39.98
21		40.04
22		39.97
23		40.02
24		39.97
25		39.96
26		39.90
27		39.92
28		39.86
29		39.91
30		39.81
31		39.87
32		39.85
33		39.90
34		39.91
35		39.90
36		39.89
37		39.87
38		39.88
39		39.87
40		39.85
41		39.84
42		39.83
43		39.83
44		39.74
45		39.78
46		39.79
47		39.79
48		39.78
49		39.78
50		39.78
51		39.77
52		39.69
53		39.72
54		39.73
55		39.73
56		39.67
57		39.69
58		39.68
59		39.67
60		39.65
61		39.64
62		39.59
63		39.61
64		39.59
65		39.58
66		39.57
67		39.56
68		39.54
69		39.45
70		39.47
71		3

**Read 1**  
![Read1](1294_S1_L008_R1_001.png)  
**Index 1**  
![Index1](1294_S1_L008_R2_001.png)  
**Read 2**  
![Read2](1294_S1_L008_R4_001.png)  
**Index 2**  
![Index2](1294_S1_L008_R3_001.png)  

### 2b) Phred Quality-Score Cut-off:  
---  
> A good quality-score cutoff often depends on the particular applicatation. A quality-score of 20 or higher is often considered usable data, because this means that only 1% of bases are likely to be erroneous. In other cases, we may desire greater certainty, in which case a quality-score cutoff of 30, meaning only 0.1% of bases are likely to be erroneous, can be a useful target. As you can see from the graph below, quality scores below 20 increase in error rates precipitously. For this reason, sequences with a quality-score below 20 are considered unreliable.   

![QscoreErrorRate](PhredQscoreErrors.png)



### 2c) Number of Indexes Containing an "N" undetermined base call:  
---  
**The following shell commands were used to retrieve this information:**  
>zcat /projects/bgmp/shared/2017_sequencing/1294_S1_L008_R2_001.fastq.gz | sed -n '2~4p' | grep -cE 'N' > index1Ncount.txt  
>zcat /projects/bgmp/shared/2017_sequencing/1294_S1_L008_R3_001.fastq.gz | sed -n '2~4p' | grep -cE 'N' > index2Ncount.txt  

_Index 1:  
**3,976,613** indexes contained an "N"_  
_Index 2:  
**3,328,051** indexes contained an "N"_




# Part 2  
---  
> Generate a strategy for developing an algorithm designed to demultiplex files and report index-hopping. That is, given four files (2 w/ biological reads, 2 w/ index reads) and known indexes, sort reads by index, outputting one forward file and one reverse file per index, plus a pair of files for unknown indexes. Additionally, your algorithm should report the number of properly matched indexes (per index) and the level of index hopping observed.  

1. Define the Problem:  
    The problem is index hopping. By chance, a certain level of index hopping is going to occur. These reads amount to
    lost data if their true identities cannot be resolved. Or, if reads are included with ambiguous identities, the 
    alignment process becomes complicated, if not confounded. The simplest approach would be to separate reads whose 
    indexes contain any ambiguity from those that have clearly matched indexes. We also need to keep tabs on all four
    files simultaneously to both determine properly matched indexes and to print each read to its appropriate file. To
    accurately count index hopping, it would be necessary to identify which index mismatches were a result of
    sequencing error and which were a result of actual index hopping. This complicates the problem, as we'll need
    to track each character in each index for rates of similarity to classify each mismatch into 3 categories: as the
    result of sequencing error, index hopping, or remaining ambiguous. Statistical methods would need to be devised to 
    validate any such process of categorization.

2. Determine/Describe What Output Would Be Informative:  
    The forward and reverse output files of properly matched indices would contain the genomic information of interest.   
    It would also be useful to know how many mismatches are due to sequencing error, how many are due to index  
    hopping, and how many are unresolvable mismatches.   
3. Write Examples (Unit Tests!):  
    i. Include four properly formatted input fastq files.  
>All Files Associated With a Unit Test End With '.fq' In the Github Folder   
    
    ii. Include the appropriate number of properly formatted output fastq files.  
    
            I created 4 indexes, so there are 4 forward read and 4 reverse read files, 
            plus 2 files for mismatches.  
4. Develop Your Algorithm Using Pseudocode:  
    I. Initialize a dictionary containing the 24 valid indices as keys, each assigned its own unique 2 string character
    IDs as values, to be used to name and identify output files.  
    
    II. Initialize an 'index hopping' counter and a 'positive match' counter.
    
    II. Open the 4 files for reading.
    
    III. For each read, check if Index 1 is in the dictionary and equal to Index 2.
>If yes, look up the index's 2 character ID in the dictionary, open 2 files for appending, with that ID +
['for' OR 'rev'] to indicate forward and reverse strands. Write the current reads from the forward and reverse
files to their own newly created file.  
  
>If no, call a comparison function to determine index similarity between index 1 and index 2. If they are
sufficiently similar, open the 2 mismatch files for appending, and append reads 1 and 2. If they are
sufficiently dissimilar, open the 2 mismatch files for appending, and append reads 1 and 2, then increment the 
index hopping counter.    
    
5. Determine High-Level Functions:  
Description  
>A high-level function to determine index similarity by comparing base-by-base between the two indices. If the 2
 indices are dissimilar, then return a Boolean True that will trigger the index-hopping counter, in-script, to
 increment.  
 
Function Headers 
>INPUT: 2 strings of nucleotides. OUTPUT: A Boolean  

Test Examples for Individual Functions  
>'ACTGACTG' compared to 'ACTGACTA' returns FALSE  
'ACTGACTG' compared to 'ACTGACCA' returns FALSE  
'ACTGACTG' compared to 'ACCAGTCA' returns TRUE  
'ACTGACTG' compared to 'CAGACAGA' returns TRUE  
2 or less differences returns FALSE.  
2 or less similarities returns TRUE.  

Return Statement  
>The function returns TRUE or FALSE, which in the script determines whether to increment the index-hopping counter.  